In [1]:
import pandas as pd

# Define los paths de los archivos Excel y los nombres de las jornadas
jugadores_df_paths = [
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX transformados\AL_J1_jugadores.xlsx',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX transformados\AL_J2_jugadores.xlsx',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX transformados\AL_J3_jugadores.xlsx',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX transformados\AL_J4_jugadores.xlsx',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX transformados\AL_J5_jugadores.xlsx',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX transformados\AL_J6_jugadores.xlsx'
]

nombres_jornadas = {
    "J1": "Jornada 1 - Local vs Universidad Cesar Vallejo",
    "J2": "Jornada 2 - Visita vs Alianza Atlético de Sullana",
    "J3": "Jornada 3 - Local vs Universitario de Deportes",
    "J4": "Jornada 4 - Visita vs Unión Comercio",
    "J5": "Jornada 5 - Local vs Comerciantes Unidos",
    "J6": "Jornada 6 - Visita vs ADT",
}

# Lista para almacenar los DataFrames de cada jornada
dfs = []

# Leer cada archivo y agregarlo a la lista de DataFrames
for path, jornada in zip(jugadores_df_paths, nombres_jornadas):
    df_temp = pd.read_excel(path)
    df_temp['Jornada'] = nombres_jornadas[jornada]  # Añade la columna 'Jornada'
    dfs.append(df_temp)

# Combinar todos los DataFrames en uno solo
df_combinado = pd.concat(dfs, ignore_index=True)

# Exportar a Excel
df_combinado.to_excel(r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX transformados\Resumen_AL_Jugadores.xlsx', index=False)


In [3]:
import pandas as pd

# Rutas a los archivos
resumen_jugadores_path = r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX finales\Resumen_AL_Jugadores.xlsx'
jugadores_2024_path = r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\ALIANZA LIMA 2024.xlsx'
a_jugadores_2024_path = r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\XLSX finales\ALIANZA LIMA 2024 AUTOMATIZADO.xlsx'

# Carga de los DataFrames
df_resumen = pd.read_excel(resumen_jugadores_path)
df_jugadores = pd.read_excel(jugadores_2024_path)

# Inicializar a 0 las columnas de acumulados antes de empezar el procesamiento
df_jugadores['Minutos'] = 0
df_jugadores['Goles'] = 0
df_jugadores['Asistencias'] = 0

# Acumular Goles y Asistencias de forma global
goles_asistencias_acumulados = df_resumen.groupby('Jugador')[['Asistencias de Gol','Goles']].sum().reset_index()

# Actualizar df_jugadores con los valores acumulados de Goles y Asistencias
for _, row in goles_asistencias_acumulados.iterrows():
    nombre_jugador = row['Jugador']
    goles = row['Goles']
    asistencias = row['Asistencias de Gol']
    idx = df_jugadores.index[df_jugadores['Jugador'] == nombre_jugador].tolist()
    if idx:
        df_jugadores.at[idx[0], 'Goles'] = goles
        df_jugadores.at[idx[0], 'Asistencias'] = asistencias


# Mapeo de jornadas detallado a corto
mapeo_jornadas = {
    "Jornada 1 - Local vs Universidad Cesar Vallejo": "J1",
    "Jornada 2 - Visita vs Alianza Atlético de Sullana": "J2",
    "Jornada 3 - Local vs Universitario de Deportes": "J3",
    "Jornada 4 - Visita vs Unión Comercio": "J4",
    "Jornada 5 - Local vs Comerciantes Unidos": "J5",
    "Jornada 6 - Visita vs ADT": "J6",
}

# Procesamiento y actualización de df_jugadores con datos de df_resumen
for jornada_detalle, jornada_corta in mapeo_jornadas.items():
    filtro_jornada = df_resumen['Jornada'].str.contains(jornada_detalle, na=False)
    df_filtrado = df_resumen[filtro_jornada]

    for _, row in df_filtrado.iterrows():
        nombre_jugador = row['Jugador']
        idx = df_jugadores.index[df_jugadores['Jugador'] == nombre_jugador].tolist()
        if idx:
            idx = idx[0]
            for col in ['Minutos Jugados']:
                valor = pd.to_numeric(row.get(col, 0), errors='coerce')
                if col == 'Minutos Jugados':
                    df_jugadores.at[idx, 'Minutos'] += valor
                    df_jugadores.at[idx, f'{jornada_corta} - Minutos'] = valor


# Guardar el DataFrame actualizado
df_jugadores.to_excel(a_jugadores_2024_path, index=False)